# Imports

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, OneCycleLR

In [ ]:
print('PyTorch version', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

In [ ]:
import os
import json
import math
import time
import random
from pathlib import Path
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

from sklearn.metrics import roc_auc_score

In [ ]:
from monai.losses import DiceLoss, FocalLoss

In [ ]:
from helper_functions import init_confmat_sums, init_auc_buffers
from helper_functions import update_confmat_sums, update_auc_buffers
from helper_functions import compute_scalar_metrics_from_confmat, compute_mean_auc

# Load the model

In [ ]:
base_model_dir = Path("Models/My_UNet_alpha_v1")

x = int(input("Choose model to load (0: dice, 1: loss, 2: epoch250): "))
chose = ["dice", "loss", "epoch250"][x]
base_model_path = base_model_dir / f"{chose}_best_model_full.pth"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(base_model_path, map_location=device)
model = model.to(device)

# Create subfolders

In [ ]:
fine_tune_index = 1

In [ ]:
fine_tune_dir = base_model_dir / f"fine-tune_{fine_tune_index}"
fine_tune_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 Fine-tuning results will be saved in: {fine_tune_dir}")

curves_dir = fine_tune_dir / "curves"
pred_dir = fine_tune_dir / "predictions"
curves_dir.mkdir(exist_ok=True)
pred_dir.mkdir(exist_ok=True)

# Parameters

#### Directories for the images

In [ ]:
parent = Path().cwd().parent
data_root = parent / 'try_1'

train_images_dir = data_root / 'train' / 'images'
train_masks_dir = data_root / 'train' / 'masks'

val_images_dir = data_root / 'val' / 'images'
val_masks_dir = data_root / 'val' / 'masks'

#### General Parameters

In [ ]:
IMG_HEIGHT = 768
IMG_WIDTH = 768

BATCH_SIZE = 4

EPOCHS = 50
print('Number of epochs:', EPOCHS)

LR = 0.001
print('Learning rate:', LR)

"""
0 : background (black)
1 : arteries (white)
2 : veins (gray)
"""
NUM_CLASSES = 3

COLOR_TO_LABEL = {
    (0,0,0): 0,
    (255,255,255): 1,
    (128,128,128): 2
}

# DataLoaders

In [ ]:
class RAVIRDataset(Dataset):
    def __init__(self, img_dir, mask_dir, img_size=(IMG_HEIGHT, IMG_WIDTH), transforms=None):
        self.img_paths = sorted(list(Path(img_dir).glob('*.png')))
        self.mask_dir = Path(mask_dir)
        self.img_size = img_size
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_p = self.img_paths[idx]
        mask_p = self.mask_dir / img_p.name
        img = Image.open(img_p).convert('RGB').resize(self.img_size, Image.BILINEAR)
        img = np.array(img, dtype=np.float32) / 255.0
        img = np.transpose(img, (2,0,1))  # C,H,W
        img_t = torch.from_numpy(img).float()
        # load mask and map colors to labels
        m = Image.open(mask_p).convert('RGB').resize(self.img_size, Image.NEAREST)
        m_arr = np.array(m, dtype=np.uint8)
        label = np.zeros((self.img_size[1], self.img_size[0]), dtype=np.uint8)
        for color, lab in COLOR_TO_LABEL.items():
            mask = np.all(m_arr == np.array(color, dtype=np.uint8), axis=-1)
            label[mask] = lab
        label_t = torch.from_numpy(label).long()  # H,W
        return img_t, label_t

In [ ]:
train_ds = RAVIRDataset(train_images_dir, train_masks_dir, (IMG_WIDTH, IMG_HEIGHT))
val_ds = RAVIRDataset(val_images_dir, val_masks_dir, (IMG_WIDTH, IMG_HEIGHT))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Function parameters

#### Optimizer

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=LR)

#### Loss function

In [ ]:
def dice_loss(outputs, targets, epsilon=1e-6):
    """
    outputs: logits (N, C, H, W)
    targets: labels (N, H, W)
    """
    # Probabilités
    probs = F.softmax(outputs, dim=1)

    N, C, H, W = probs.shape

    # One-hot du masque → shape (N, C, H, W)
    targets_onehot = F.one_hot(targets, num_classes=C).permute(0, 3, 1, 2).float()

    # On ignore la classe 0 (background)
    probs = probs[:, 1:, :, :]          # (N, C-1, H, W)
    targets_onehot = targets_onehot[:, 1:, :, :]  # (N, C-1, H, W)

    # Dice par classe
    intersection = torch.sum(probs * targets_onehot, dim=(0, 2, 3))
    cardinality  = torch.sum(probs + targets_onehot, dim=(0, 2, 3))

    dice_per_class = (2 * intersection + epsilon) / (cardinality + epsilon)

    # Loss = 1 - moyenne des dice des classes artery + vein
    return 1 - dice_per_class.mean()

In [ ]:
percentage = [0.872, 0.054, 0.073]
inverse_percentage = [round((1/p),2) for p in percentage]
print(inverse_percentage)

weights = torch.tensor(inverse_percentage).to(device)
CE_loss = nn.CrossEntropyLoss(weight=weights)

##### Alpha functions

In [ ]:
def alpha_linear(epoch, total_epochs, start=0.9, end=0.1):
    """α(t) linearly decrease from start to end"""
    return start - (start - end) * (epoch / (total_epochs - 1))

In [ ]:
def alpha_exponential(epoch, total_epochs, start=0.9, end=0.1, k=5) :
	"""α(t) decreases fastly at first, then stabilises"""
	return end + (start-end)*np.exp(-k * epoch / (total_epochs-1))

#### Scheduler

In [ ]:
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR*10, 
    epochs=EPOCHS, 
    steps_per_epoch=len(train_loader)
)

# Fine-tunning

In [ ]:
NUM_CLASSES = 3
BACKGROUND_IDX = 0
VESSEL_CLASSES = [1, 2]

In [ ]:
best_val_loss = float("inf")
best_val_dice = -float("inf")

save_dir = fine_tune_dir

In [ ]:
train_history = {
    "loss": [],
    "accuracy":   [],
    "dice":       [],
    "iou":        [],
    "precision":  [],
    "recall":     [],
    "sensitivity":[],
    "specificity":[],
    "auc":        [],
    "lr":         [],
    "alpha":      []
}

val_history = {
    "loss": [],
    "accuracy":   [],
    "dice":       [],
    "iou":        [],
    "precision":  [],
    "recall":     [],
    "sensitivity":[],
    "specificity":[],
    "auc":        []
}

In [ ]:
# =========================
#        LEARNING
# =========================

for epoch in range(EPOCHS) :

    # =========================
    #        TRAIN
    # =========================

    model.train()

    runing_loss = 0.0
    conf_sums = init_confmat_sums()
    auc_buf = init_auc_buffers()

    for imgs, masks in train_loader :

        imgs = imgs.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)

        alpha = alpha_exponential(epoch, EPOCHS, start=0.9, end=0.1, k=5)
        loss = alpha * CE_loss(outputs, masks) + (1-alpha) * dice_loss(outputs, masks)
        loss.backward()

        optimizer.step()
        scheduler.step()

        runing_loss += loss.item() * imgs.size(0)

        preds = outputs.argmax(dim=1)
        update_confmat_sums(conf_sums, preds, masks, NUM_CLASSES)
        update_auc_buffers(auc_buf, outputs, masks)
    
    train_loss = runing_loss / len(train_ds)
    train_metrics = compute_scalar_metrics_from_confmat(conf_sums, VESSEL_CLASSES)
    train_auc = compute_mean_auc(auc_buf)

    train_history["loss"].append(train_loss)
    train_history["accuracy"].append(train_metrics["accuracy"])
    train_history["dice"].append(train_metrics["dice"])
    train_history["iou"].append(train_metrics["iou"])
    train_history["precision"].append(train_metrics["precision"])
    train_history["recall"].append(train_metrics["recall"])
    train_history["specificity"].append(train_metrics["specificity"])
    train_history["auc"].append(train_auc)
    train_history["lr"].append(scheduler.get_last_lr()[0])
    train_history["alpha"].append(alpha)




    # =========================
    #       VALIDATION
    # =========================
    model.eval()

    val_runing_loss = 0.0
    conf_sums_val = init_confmat_sums()
    auc_buf_val   = init_auc_buffers()

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs  = imgs.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)

            outputs = model(imgs)

            alpha = alpha_exponential(epoch, EPOCHS, start=0.9, end=0.1, k=5)
            loss = alpha * CE_loss(outputs, masks) + (1-alpha) * dice_loss(outputs, masks)

            val_runing_loss += loss.item() * imgs.size(0)

            preds = outputs.argmax(dim=1)
            update_confmat_sums(conf_sums_val, preds, masks, NUM_CLASSES)
            update_auc_buffers(auc_buf_val, outputs, masks)

    val_loss = val_runing_loss / len(val_ds)
    val_metrics = compute_scalar_metrics_from_confmat(conf_sums_val, VESSEL_CLASSES)
    val_auc = compute_mean_auc(auc_buf_val)

    val_history["loss"].append(val_loss)
    val_history["accuracy"].append(val_metrics["accuracy"])
    val_history["dice"].append(val_metrics["dice"])
    val_history["iou"].append(val_metrics["iou"])
    val_history["precision"].append(val_metrics["precision"])
    val_history["recall"].append(val_metrics["recall"])
    val_history["specificity"].append(val_metrics["specificity"])
    val_history["auc"].append(val_auc)




    # =========================
    #           LOGS
    # =========================
    print('-'*30)
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f} | Dice: {train_metrics['dice']:.4f} | IoU: {train_metrics['iou']:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Dice: {val_metrics['dice']:.4f} | IoU: {val_metrics['iou']:.4f}")




    # =========================
    #     SAVE BEST MODEL
    # =========================
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), str(save_dir / "loss_best_model_state_dict.pth"))
        torch.save(model, str(save_dir / "loss_best_model_full.pth"))
        print(f"✅ Nouveau meilleur modèle loss sauvegardé")
    
    val_dice = val_metrics["dice"]
    if val_dice > best_val_dice :
        best_val_dice = val_dice
        torch.save(model.state_dict(), str(save_dir / "dice_best_model_state_dict.pth"))
        torch.save(model, str(save_dir / "dice_best_model_full.pth"))
        print(f"✅ Nouveau meilleur modèle dice sauvegardé")



# =========================
#     SAVE LAST MODEL
# =========================
torch.save(model.state_dict(), str(save_dir / f"epoch{EPOCHS}_model_state_dict.pth"))
torch.save(model, str(save_dir / f"epoch{EPOCHS}_model_full.pth"))

# Validation tests

#### Load the best model

In [ ]:
model = torch.load(save_dir / f"{x}_best_model_full.pth")

#### Images

In [ ]:
model.eval()
total_correct = 0
total_pixels = 0

# 📁 Dossier pour sauvegarder les prédictions
pred_dir = save_dir / "predictions"
pred_dir.mkdir(parents=True, exist_ok=True)

with torch.no_grad():
    img_index = 0  # compteur global des images

    for imgs, masks in val_loader:
        imgs = imgs.to(device)
        masks = masks.to(device)

        outputs = model(imgs)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        # Boucle sur chaque image du batch
        for i in range(len(imgs)):
            pred = preds[i]
            mask = masks[i]

            # ===== Calcul de l’accuracy de cette image =====
            correct_pixels = (pred == mask).sum().item()
            total_pixels_img = mask.numel()
            acc_img = correct_pixels / total_pixels_img

            print(f"🖼️ Image {img_index+1} — Accuracy : {acc_img:.4f}")

            # ===== Création de la figure =====
            fig, ax = plt.subplots(1, 3, figsize=(10, 4))
            ax[0].imshow(imgs[i].cpu().permute(1, 2, 0))
            ax[0].set_title("Original picture")
            ax[1].imshow(mask.cpu(), cmap="gray")
            ax[1].set_title("Real mask")
            ax[2].imshow(pred.cpu(), cmap="gray")
            ax[2].set_title("Predicted mask")
            for a in ax:
                a.axis("off")
            plt.tight_layout()

            # 📌 Sauvegarde dans Models/<model_name>/predictions/
            save_path = pred_dir / f"val_img_{img_index:03d}.png"
            plt.savefig(save_path, dpi=150, bbox_inches='tight')

            # ✅ Affichage immédiat dans notebook
            plt.show()

            # ===== Accumulateur global =====
            total_correct += correct_pixels
            total_pixels += total_pixels_img
            img_index += 1

# ===== Accuracy globale =====
val_acc = total_correct / total_pixels
print(f"\n✅ Accuracy moyenne sur le jeu de validation : {val_acc:.4f}")

#### Curves

In [ ]:
def plot_metric(train_history, val_history, key, title=None, mark_best_on="val_loss", save_folder=curves_dir):
    """
    key:        nom de la métrique dans les dicts (ex: "loss", "dice", "auc")
    mark_best_on:
        - "val_loss" → marquer l'epoch où val_loss est minimal
        - "val_<key>" → marquer la meilleure valeur de cette métrique (max)
    """

    train = train_history.get(key, [])
    val   = val_history.get(key, [])

    if len(val) == 0:
        print(f"⚠️ Impossible de tracer '{key}' (pas de valeurs validation trouvées)")
        return

    epochs = range(1, len(val)+1)

    plt.figure(figsize=(7,5))
    plt.plot(epochs, train, label=f"Train {key}", linewidth=2)
    plt.plot(epochs, val,   label=f"Val {key}", linewidth=2)

    # ========= Marqueur du "best" =========
    if mark_best_on == "val_loss" and "loss" in val_history:
        best_e = int(np.argmin(val_history["loss"])) + 1
        best_v = val[best_e-1]
        plt.scatter([best_e], [best_v], s=80, marker="o", color="red", zorder=3,
                    label=f"best @ epoch {best_e}")
    elif mark_best_on == f"val_{key}" and len(val) > 0:
        best_e = int(np.nanargmax(val)) + 1
        best_v = val[best_e-1]
        plt.scatter([best_e], [best_v], s=80, marker="o", color="red", zorder=3,
                    label=f"best @ epoch {best_e}")

    plt.xlabel("Epoch")
    plt.ylabel(key)
    plt.title(title or f"{key} (train vs val)")
    plt.grid(True)
    plt.legend()

    # ========= SAVE =========
    save_path = save_folder / f"{key}.png"
    plt.savefig(save_path, dpi=200, bbox_inches="tight")
    print(f"💾 Courbe sauvegardée : {save_path}")

    plt.show()   # continue à afficher


# ======== TRAÇAGE ========

plot_metric(train_history, val_history, "loss", title="Loss curves", mark_best_on="val_loss")
plot_metric(train_history, val_history, "dice", title="Dice (artery+vein avg)", mark_best_on="val_dice")
plot_metric(train_history, val_history, "auc",  title="AUC (artery+vein avg)",  mark_best_on="val_auc")

In [ ]:
alpha_values = train_history.get("alpha", [])
lr_values = train_history.get("lr", [])
epochs = np.arange(1, len(alpha_values) + 1)


# =========================
#   COURBE ALPHA(t)
# =========================
fig_alpha, ax = plt.subplots(figsize=(8, 5))
ax.plot(epochs, alpha_values, color="#1f77b4", linewidth=2)
ax.set_title("Evolution of α in Loss = α * CE + (1-α)*DiceLoss")
ax.set_xlabel("Epoch", fontsize=11)
ax.set_ylabel("α value", fontsize=11)
ax.grid(alpha=0.4)
ax.legend(["α(t) schedule"], loc="best")
plt.tight_layout()

alpha_path = curves_dir / "alpha_curve.png"
plt.savefig(alpha_path, dpi=200, bbox_inches='tight')
print(f"✅ Courbe α(t) sauvegardée dans : {alpha_path}")

plt.show()



# =========================
#   COURBE LEARNING RATE
# =========================
fig_lr, ax = plt.subplots(figsize=(8, 5))
ax.plot(epochs, lr_values, color="#ff7f0e", linewidth=2)
ax.set_title("Learning Rate Schedule")
ax.set_xlabel("Epoch", fontsize=11)
ax.set_ylabel("Learning rate", fontsize=11)
ax.grid(alpha=0.4)
ax.legend(["lr(t)"], loc="best")
plt.tight_layout()

lr_path = curves_dir / "lr_curve.png"
plt.savefig(lr_path, dpi=200, bbox_inches='tight')
print(f"✅ Courbe learning rate sauvegardée dans : {lr_path}")

plt.show()

#### Summary of the metrics

In [ ]:
# =========================
#   CONFIG CLASSES
# =========================
ARTERY_CLASS = 1
VEIN_CLASS   = 2
VESSEL_CLASSES = [ARTERY_CLASS, VEIN_CLASS]
assert NUM_CLASSES >= 3, "NUM_CLASSES doit inclure background + artery + vein"


# =========================
#  UTILS: métriques par classe
# =========================
def compute_per_class_metrics_from_confmat(sums, c):
    TP, FP, FN, TN = sums["TP"][c], sums["FP"][c], sums["FN"][c], sums["TN"][c]
    eps = 1e-7

    acc  = (TP + TN) / max(TP + TN + FP + FN, eps)
    prec = TP / max(TP + FP, eps)
    rec  = TP / max(TP + FN, eps)
    spec = TN / max(TN + FP, eps)
    dice = (2 * TP) / max(2 * TP + FP + FN, eps)
    iou  = TP / max(TP + FP + FN, eps)

    return {
        "accuracy":   float(acc),
        "precision":  float(prec),
        "recall":     float(rec),      # Sensitivity
        "specificity":float(spec),
        "dice":       float(dice),
        "iou":        float(iou),
    }

def compute_auc_for_class(buffers, c):
    pack = buffers.get(c, None)
    if pack is None or len(pack["y_true"]) == 0:
        return float("nan")

    y_true  = np.concatenate(pack["y_true"])
    y_score = np.concatenate(pack["y_score"])

    pos = (y_true == 1).sum()
    neg = (y_true == 0).sum()
    if pos == 0 or neg == 0:
        return float("nan")

    try:
        return float(roc_auc_score(y_true, y_score))
    except:
        return float("nan")


# =========================
#   ÉVALUATION
# =========================
model.eval()
conf_sums_val = init_confmat_sums(num_classes=NUM_CLASSES)
auc_buf_val   = init_auc_buffers(include_classes=VESSEL_CLASSES)

with torch.no_grad():
    for imgs, masks in val_loader:
        imgs  = imgs.to(device)
        masks = masks.to(device)

        outputs = model(imgs)
        preds = outputs.argmax(dim=1)

        update_confmat_sums(conf_sums_val, preds, masks, NUM_CLASSES)
        update_auc_buffers(auc_buf_val, outputs, masks)


# =========================
#   MÉTRIQUES PAR CLASSE
# =========================
results = {}

for cls, name in zip(VESSEL_CLASSES, ["Artery", "Vein"]):
    m = compute_per_class_metrics_from_confmat(conf_sums_val, cls)
    m["auc"] = compute_auc_for_class(auc_buf_val, cls)
    results[name] = m


# =========================
#   MOYENNES ARTERY + VEIN
# =========================
avg_metrics = {metric: np.nanmean([results["Artery"][metric], results["Vein"][metric]])
               for metric in results["Artery"].keys()}

results["Average (A+V)"] = avg_metrics


# =========================
#   AFFICHAGE TABLEAU (4 décimales)
# =========================
df = pd.DataFrame(results).T
df = df.round(4)  # <= Arrondi propre à 4 décimales

print("\n=== Résultats de validation ===\n")
display(df.style.format("{:.4f}"))


In [ ]:
# =========================
#   CONFIG CLASSES
# =========================
ARTERY_CLASS = 1
VEIN_CLASS   = 2
VESSEL_CLASSES = [ARTERY_CLASS, VEIN_CLASS]
assert NUM_CLASSES >= 3, "NUM_CLASSES must include background + artery + vein"

# =========================
#   PER-IMAGE METRIC HELPERS
# =========================
def dice_per_image(pred, target, cls):
    eps = 1e-7
    p = (pred == cls).float()
    t = (target == cls).float()
    inter = (p * t).sum()
    denom = p.sum() + t.sum()
    return float((2.0 * inter) / (denom + eps))

def iou_per_image(pred, target, cls):
    eps = 1e-7
    p = (pred == cls)
    t = (target == cls)
    inter = (p & t).sum().item()
    union = (p | t).sum().item()
    return float(inter / (union + eps))

def auc_per_image_from_logits(logits, target, cls):
    # logits: (C,H,W) for a single image
    probs = F.softmax(logits, dim=0)
    y_score = probs[cls].reshape(-1).cpu().numpy()
    y_true  = (target == cls).reshape(-1).cpu().numpy().astype(np.uint8)
    pos = (y_true == 1).sum()
    neg = (y_true == 0).sum()
    if pos == 0 or neg == 0:
        return float("nan")
    try:
        return float(roc_auc_score(y_true, y_score))
    except Exception:
        return float("nan")

# =========================
#   COLLECT PER-IMAGE METRICS
# =========================
model.eval()

artery_dice_list, vein_dice_list, avg_dice_list = [], [], []
artery_iou_list,  vein_iou_list,  avg_iou_list  = [], [], []
artery_auc_list,  vein_auc_list,  avg_auc_list  = [], [], []

with torch.no_grad():
    for imgs, masks in val_loader:
        imgs  = imgs.to(device)
        masks = masks.to(device)

        logits = model(imgs)          # (N,C,H,W)
        preds  = logits.argmax(dim=1) # (N,H,W)

        for i in range(imgs.size(0)):
            p   = preds[i]
            gt  = masks[i]
            log = logits[i]

            # Dice
            a_d = dice_per_image(p, gt, ARTERY_CLASS)
            v_d = dice_per_image(p, gt, VEIN_CLASS)
            artery_dice_list.append(a_d)
            vein_dice_list.append(v_d)
            avg_dice_list.append(np.nanmean([a_d, v_d]))

            # IoU
            a_i = iou_per_image(p, gt, ARTERY_CLASS)
            v_i = iou_per_image(p, gt, VEIN_CLASS)
            artery_iou_list.append(a_i)
            vein_iou_list.append(v_i)
            avg_iou_list.append(np.nanmean([a_i, v_i]))

            # AUC
            a_auc = auc_per_image_from_logits(log, gt, ARTERY_CLASS)
            v_auc = auc_per_image_from_logits(log, gt, VEIN_CLASS)
            artery_auc_list.append(a_auc)
            vein_auc_list.append(v_auc)
            avg_auc_list.append(np.nanmean([a_auc, v_auc]))

# =========================
#   BAR PLOTS (3 rows × 3 cols)
#   One bar per image; red dashed = mean
# =========================
def plot_bars_one_metric(ax, values, title):
    values = np.array(values, dtype=float)
    n = len(values)
    x = np.arange(1, n+1)

    ax.bar(x, values, color="#1f77b4", edgecolor="black", alpha=0.9)

    m = np.nanmean(values)
    ax.axhline(m, color="red", linestyle="--", linewidth=2, label=f"mean = {m:.3f}")

    ax.set_ylim(0, 1)            # metrics in [0,1]
    ax.set_xlim(0.5, n + 0.5)
    ax.set_xlabel("Validation image index")
    ax.set_ylabel("Score")
    ax.set_title(title)
    # lighten x ticks if many images
    if n > 30:
        ax.set_xticks(np.linspace(1, n, 10, dtype=int))
    ax.grid(axis="y", alpha=0.3)
    ax.legend(loc="upper right")

fig, axes = plt.subplots(3, 3, figsize=(20, 12), constrained_layout=True)

# Row 1: Dice
plot_bars_one_metric(axes[0,0], artery_dice_list, "Dice – Artery")
plot_bars_one_metric(axes[0,1], vein_dice_list,   "Dice – Vein")
plot_bars_one_metric(axes[0,2], avg_dice_list,    "Dice – Average (A+V)")

# Row 2: IoU
plot_bars_one_metric(axes[1,0], artery_iou_list, "IoU – Artery")
plot_bars_one_metric(axes[1,1], vein_iou_list,   "IoU – Vein")
plot_bars_one_metric(axes[1,2], avg_iou_list,    "IoU – Average (A+V)")

# Row 3: AUC
plot_bars_one_metric(axes[2,0], artery_auc_list, "AUC – Artery")
plot_bars_one_metric(axes[2,1], vein_auc_list,   "AUC – Vein")
plot_bars_one_metric(axes[2,2], avg_auc_list,    "AUC – Average (A+V)")

plt.show()